NOTE: DO NOT RUN THIS CODE IF NODELISTS_DIR and EDGELISTS_DIR are already populated. You will overwrite the already preprocessed data.

This code is used to convert the raw partek scRNA-Seq files which contain thousands of cells and their respective gene information into files containing the data for each cell's protein-protein interaction network.

Each protein-protein interaction network data is represented as a nodelist and edgelist. The nodelist contains all the present nodes and their corresponding weight (as determined from the node's gene expression level). The edgelist contains all the links between the nodes in the nodelist.

In [3]:
import os
import pandas as pd
import numpy as np
import math

After importing the libraries, the first step is to declare which directory contains your raw data, and what filenames in the raw data directory you would like to work with.

In addition, declare where you will output your nodelists and edgelists.

In [4]:
RAW_DATA_DIR = 'raw_data/'
RAW_FILENAMES = ['onlyhallmarks_COUNT_MATRIX.txt']
#SUFFIXES = ['gerGFP','gerKL', 'oldGFP', 'oldKL','youngGFP','youngSham']

GENE_EXPR_DIR = 'gene_expression/'
NODELISTS_DIR = 'nodelists/'
EDGELISTS_DIR = 'edgelists/'

Now we have to check that all the cells have a unique name as we'll be  naming the output nodelist and edglist based on the cell's name and we don't want any overlap.

In [5]:
# check for uniqueness here

def removeDuplicates(listofElements):
    
    # Create an empty list to store unique elements
    uniqueList = []
    duplicates = []
    
    # Iterate over the original list and for each element
    # add it to uniqueList, if its not already there.
    for elem in listofElements:
        if elem not in uniqueList:
            uniqueList.append(elem)
        else:
            duplicates.append(elem)
    
    # Return the list of unique elements        
    return uniqueList, duplicates

Our first step of the preprocessing will be to define a function to convert our large raw data input containing many cells into files where each file contains the gene expression for a single cell.

The optional suffix parameter is for labeling cell files with a particular group label such as "old" or "ActivatedMuSCs." For this run, we will use suffix to distinguish between age groups.

In [9]:
CHUNK_SIZE = 1000 # number of rows to read into memory at a time

def buildGeneExpressionFile(raw_file, suffix='', 
                             show_progress=True):
    listofElements=[]
    
    if show_progress: 
        print('Building from ' + raw_file)
        chunk_count = 1
    
    # create spacers around the suffix if not empty
    if suffix: suffix = '_' + suffix + '_'
    
    for chunk in pd.read_csv(raw_file,',',chunksize=CHUNK_SIZE):
        # drop the non-gene columns and set the index to be
        # our cell ID
        
        #chunk.drop(chunk.columns[1:8],axis=1,inplace=True)
        chunk.set_index('rn',inplace=True)
        
        if show_progress: 
            print('On chunk ' + str(chunk_count))
            chunk_count+=1
        
        # iterate over all the cells in our chunk
        for idx in chunk.index:
            
            #Store all cell ids to a list -for uniqueness testing
            listofElements.append(str(idx))
            
            # get our cell data
            row = chunk.loc[idx]
            
            # create our cell gene expression file
            filename = str(idx)+'_gene_symbol'+suffix+'.csv'
            gene_file = os.path.join(GENE_EXPR_DIR,filename)
            
            # write to our file
            with open(gene_file,'w') as f:      
                for gene in row.index:
                    f.write(gene+','+str(row[gene])+'\n') 
                    
    uniqueList, duplicates=removeDuplicates(listofElements)
    
    print("Length of listofElements: %s"%len(listofElements))
    print("Length of uniqueList: %s"%len(uniqueList))
    print("Length of duplicates: %s"%len(duplicates))

In [10]:
def buildAllGeneExpressionFiles():

    for idx, filename in enumerate(RAW_FILENAMES):
        raw_file = os.path.join(RAW_DATA_DIR,filename)
        #suffix = SUFFIXES[idx]
        suffix='FA'
        buildGeneExpressionFile(raw_file,suffix=suffix)

In [11]:
# Uncomment to run
buildAllGeneExpressionFiles()

Building from raw_data/onlyhallmarks_COUNT_MATRIX.txt
On chunk 1
Length of listofElements: 24
Length of uniqueList: 24
Length of duplicates: 0


We now have to convert each of our gene IDs into their corresponding transcribed proteins. This is done with our MOUSE_ENSEMBL_ID_MAP_FILE which was sourced from The Jackson Laboratory's MGI resource.

In [12]:
MOUSE_ENSEMBL_ID_MAP_FILE = 'mouse_ensembl_id_map.csv'

We will store the file's data into memory as a dictionary (which works fine because the file is small < 5 MB) for quick O(1) search for the corresponding protein IDs.

In [13]:
df = pd.read_csv(MOUSE_ENSEMBL_ID_MAP_FILE)
#df = df.groupby('gene_symbol')['ensembl_id'].apply(list)
df = df.groupby('gene_id')['ensembl_id'].apply(list)
MOUSE_ENSEMBL_ID_MAP = df.to_dict()

In [14]:
MOUSE_ENSEMBL_ID_MAP

{'ENSMUSG00000000001': ['ENSMUSP00000000001'],
 'ENSMUSG00000000003': ['ENSMUSP00000000003', 'ENSMUSP00000109675'],
 'ENSMUSG00000000028': ['ENSMUSP00000000028',
  'ENSMUSP00000094753',
  'ENSMUSP00000111248'],
 'ENSMUSG00000000037': ['ENSMUSP00000158772',
  'ENSMUSP00000098672',
  'ENSMUSP00000084325',
  'ENSMUSP00000074356',
  'ENSMUSP00000019101',
  'ENSMUSP00000076593',
  'ENSMUSP00000158809',
  'ENSMUSP00000107964'],
 'ENSMUSG00000000049': ['ENSMUSP00000000049',
  'ENSMUSP00000115516',
  'ENSMUSP00000123486',
  'ENSMUSP00000114214'],
 'ENSMUSG00000000056': ['ENSMUSP00000099304'],
 'ENSMUSG00000000058': ['ENSMUSP00000000058',
  'ENSMUSP00000111119',
  'ENSMUSP00000111122'],
 'ENSMUSG00000000078': ['ENSMUSP00000000080', 'ENSMUSP00000152088'],
 'ENSMUSG00000000085': ['ENSMUSP00000000087',
  'ENSMUSP00000101908',
  'ENSMUSP00000115343',
  'ENSMUSP00000120950',
  'ENSMUSP00000101905',
  'ENSMUSP00000069813'],
 'ENSMUSG00000000088': ['ENSMUSP00000000090'],
 'ENSMUSG00000000093': ['ENSMU

We are defining a function to be used across every file. This function will keep passing a dictionary that will keep track of all the genes seen so far. If a gene in the file does not appear in our dictionary, we will search for the gene in MOUSE_ENSEMBL_ID_MAP and add it to our dictionary to be used for all future files (because the files all share a similar set of genes - same within each raw file but slight differences between raw files). We use a dictionary because of its O(1) look up time.

In [15]:
def buildNodelist(gene_expression_file):

    # get nodelist filename from input
    nodelist = os.path.basename(gene_expression_file)
    nodelist = nodelist.replace('gene_symbol','nodelist')
    nodelist = os.path.join(NODELISTS_DIR, nodelist)
    
    # read input
    df = pd.read_csv(gene_expression_file,
                     header=None,
                     names=['gene','val'],
                     index_col='gene')
    #print(df)
    
    # write all protein nodes and weights to file
    with open(nodelist,'w') as f:
        for gene in df.index:
            if gene in MOUSE_ENSEMBL_ID_MAP:
                if float(df['val'][gene]) > 0:
                #if MOUSE_ENSEMBL_ID_MAP[gene]['val'] > 0:
                #if MOUSE_ENSEMBL_ID_MAP[gene].['val'] > 0:
                    for protein in MOUSE_ENSEMBL_ID_MAP[gene]:
                        f.write(protein+','+str(df['val'][gene])+'\n')

In [16]:
def buildAllNodelists(show_progress=True):
    
    total = len(os.listdir(GENE_EXPR_DIR))

    if show_progress: print('Total: ' + str(total) + ' files')

    count = 0
        
    for file in os.listdir(GENE_EXPR_DIR):
        gene_expression_file = os.path.join(GENE_EXPR_DIR,file)
        buildNodelist(gene_expression_file)
        count+=1
        if count % 1 == 0:
            print(str(count) + '/' + str(total) + ' completed')

In [17]:
# Uncomment to run
buildAllNodelists()

Total: 24 files
1/24 completed
2/24 completed
3/24 completed
4/24 completed
5/24 completed
6/24 completed
7/24 completed
8/24 completed
9/24 completed
10/24 completed
11/24 completed
12/24 completed
13/24 completed
14/24 completed
15/24 completed
16/24 completed
17/24 completed
18/24 completed
19/24 completed
20/24 completed
21/24 completed
22/24 completed
23/24 completed
24/24 completed


Make our interactions file smaller by eliminating unnecessary columns and removing the '10090.' mouse taxon prefix, so we can more easily load it into memory and compare it to our nodelists.

In [18]:
RAW_PPI_FILE = 'raw_ppi_interactions.csv'
CLEAN_PPI_FILE = 'clean_ppi_interactions.csv'

In [45]:
# Takes about 15 seconds to run
def cleanPPIFile():
    for chunk in pd.read_csv(RAW_PPI_FILE, 
                             chunksize=10**6, 
                             usecols=['to','from']):
        chunk['from'] = chunk['from'].apply(
            lambda x: x[6:] if x.startswith('10090.') else x)
        chunk['to'] = chunk['to'].apply(
            lambda x: x[6:] if x.startswith('10090.') else x)
        chunk.to_csv(CLEAN_PPI_FILE, mode='a', index=False)

# Uncomment below line to run
#cleanPPIFile()

We will load all our PPI interactions into memory in both directions.

In [19]:
# get the adjacency lists in both directions
df = pd.read_csv(CLEAN_PPI_FILE)
PPI_ADJ_LIST = \
    df.groupby('from')['to'].apply(list).to_dict()
PPI_ADJ_LIST_REV = \
    df.groupby('to')['from'].apply(list).to_dict()

Given a single nodelist, build the edgelist from our PPI interaction mappings. 

In [20]:
def buildEdgelist(nodelist):
    
    # get edgelist filename
    edgelist = os.path.basename(nodelist)
    edgelist = edgelist.replace('nodelist','edgelist')
    edgelist = os.path.join(EDGELISTS_DIR,edgelist)
    
    # read nodelist
    df = pd.read_csv(nodelist,
                     header=None,
                     names=['gene','val'],
                     index_col='gene')
    
    # only consider nodes with a positive expresssion
    df = df[df['val']>0]
    
    # write to edgelist
    with open(edgelist,'w') as f:
        for gene in df.index:
            if gene in PPI_ADJ_LIST:
                for neighbor in PPI_ADJ_LIST[gene]:
                    if neighbor in df.index:
                        f.write(gene+','+neighbor+'\n')
            if gene in PPI_ADJ_LIST_REV:
                for neighbor in PPI_ADJ_LIST_REV[gene]:
                    if neighbor in df.index:
                        f.write(neighbor+','+gene+'\n')

Function to iterate through all nodelists and build the corresponding edgelists. (Note: the combined size of all your edgelist files will be large (>50GB) if you have 1000+ cells)

In [21]:
from tqdm import tqdm

def buildAllEdgelists(show_progress=True):
    
    total = len(os.listdir(NODELISTS_DIR))

    if show_progress: print('Total: ' + str(total) + ' files')

    count = 0
        
    for file in tqdm(os.listdir(NODELISTS_DIR)):
        nodelist = os.path.join(NODELISTS_DIR,file)
        buildEdgelist(nodelist)
        count+=1
        if count % 2 == 0:
            print(str(count) + '/' + str(total) + ' completed')
    


In [22]:
# Uncomment to run
buildAllEdgelists()

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Total: 24 files


  8%|██████▉                                                                            | 2/24 [00:10<01:59,  0.18it/s]

2/24 completed


 17%|█████████████▊                                                                     | 4/24 [00:20<01:44,  0.19it/s]

...

 83%|████████████████████████████████████████████████████████████████████▎             | 20/24 [01:44<00:20,  0.19it/s]

20/24 completed


 92%|███████████████████████████████████████████████████████████████████████████▏      | 22/24 [01:54<00:10,  0.19it/s]

22/24 completed


24/24 completed
